# 标记五官

In [82]:
import numpy as np
from collections import Counter
from PIL import Image,ImageDraw
import face_recognition
import colorsys
import PIL.Image as Image
import json

In [83]:

path="D:\\homework\\大三下信息系统设计\\test9.jpg"
image = face_recognition.load_image_file(path)
# Find all the faces in the image
print(type(image))
face_locations = face_recognition.face_locations(image)
print(face_locations)

# Or maybe find the facial features in the image
face_landmarks_list = face_recognition.face_landmarks(image)
print(face_landmarks_list)

# Or you could get face encodings for each face in the image:

list_of_face_encodings = face_recognition.face_encodings(image)

<class 'numpy.ndarray'>
[(194, 553, 657, 91)]
[{'chin': [(157, 370), (160, 420), (172, 470), (194, 523), (216, 574), (240, 626), (266, 677), (295, 721), (341, 738), (401, 730), (466, 698), (527, 656), (584, 605), (621, 543), (636, 469), (639, 391), (638, 315)], 'left_eyebrow': [(129, 293), (144, 279), (167, 282), (193, 292), (219, 304)], 'right_eyebrow': [(304, 291), (350, 271), (399, 259), (450, 267), (497, 286)], 'nose_bridge': [(262, 348), (256, 392), (249, 436), (241, 481)], 'nose_tip': [(237, 506), (251, 516), (269, 522), (288, 515), (310, 508)], 'left_eye': [(164, 347), (178, 325), (209, 326), (237, 351), (207, 363), (178, 364)], 'right_eye': [(346, 347), (368, 318), (407, 318), (442, 336), (412, 355), (375, 357)], 'top_lip': [(243, 597), (243, 570), (260, 556), (279, 562), (300, 553), (344, 568), (391, 593), (374, 592), (306, 586), (285, 588), (266, 587), (253, 595)], 'bottom_lip': [(391, 593), (355, 625), (320, 637), (296, 639), (273, 635), (256, 618), (243, 597), (253, 595), (

In [84]:
pos = face_locations[0]
# pil_image.save("quick4.jpg")

# 裁剪出脸部区域

In [85]:

pil_image =Image.open(path)
#pil_image.show()
print(type(pil_image),pil_image.size)
cropped_face = pil_image.crop((pos[3], pos[0], pos[1], pos[2])) # (left, upper, right, lower) 左上，右下
Image._show(cropped_face)
print(type(cropped_face),cropped_face.size)


<class 'PIL.JpegImagePlugin.JpegImageFile'> (665, 1024)
<class 'PIL.Image.Image'> (462, 463)


In [86]:
# 先将嘴部区域选为白色，绘制图片，然后将图片中脸的区域裁剪出来

In [87]:
pil_image = Image.fromarray(image)
a=pil_image.size
print(a,type(a))
blank_mouse = Image.new('RGB', (a[0],a[1]), (0, 0, 0))
for face_landmarks in face_landmarks_list:
    d = ImageDraw.Draw(blank_mouse, 'RGBA')
    # Gloss the lips
    d.polygon(face_landmarks['top_lip'], fill=(255,255,255,255))
    d.polygon(face_landmarks['bottom_lip'], fill=(255, 255,255, 255))
    d.line(face_landmarks['top_lip'], fill=(255, 255,255, 255), width=1)
    d.line(face_landmarks['bottom_lip'], fill=(225, 255, 255,255), width=1)
    #blank_mouse.show()
blank_mouse=blank_mouse.crop((pos[3], pos[0], pos[1], pos[2])) # (left, upper, right, lower) 左上，右下
Image._show(blank_mouse)

(665, 1024) <class 'tuple'>


In [88]:
bottom_lip=face_landmarks_list[0][ 'bottom_lip']

In [89]:
def masklayer(origin,mask):
    mask1=np.array(mask)
    print(mask1.shape)
    origin1=np.array(origin)
    print(origin1.shape)
    for i in range(len(mask1)):
        for j in range(len(mask1[i])):
   #          print(mask1[i][j][1])
            if (mask1[i][j][1]>=128):#lip
                pass
            else:#others
                origin1[i][j]=[0,0,0]
    new_png = Image.fromarray(origin1)
    new_png.show()
    #new_png.save('C:/Users/MCY/Desktop/test7.JPG')
    return new_png
cropped=masklayer(cropped_face,blank_mouse)

(463, 462, 3)
(463, 462, 3)


# 查看主要出现的颜色

In [95]:

def get_dominant_color(image):
    #颜色模式转换，以便输出rgb颜色值
    image = image.convert('RGB')
     
    #生成缩略图，减少计算量，减小cpu压力
    image.thumbnail((200, 200))
    max_score = 0
    dominant_color = 0
    for count,(r,g,b) in image.getcolors(image.size[0]*image.size[1]):
        if (r<100) :
            continue
        # 转为HSV标准
        saturation = colorsys.rgb_to_hsv(r/255.0, g/255.0, b/255.0)[1]
        y = min(abs(r*2104+g*4130+b*802+4096+131072)>>13,235)
        y = (y-16.0)/(235-16)
  
        #忽略高亮色
        if y > 0.9:
            continue
        score = (saturation+0.1)*count
        if score > max_score:
            max_score = score
            dominant_color = (r,g,b)
    return dominant_color

# 在数据库中查找对应的唇膏颜色

In [96]:
import string
def hex2rgb(hexarray,RGB_array,len_sum):
    value_array=np.zeros((sum,6),int)
    for i in range(lensum):
        for l in range(6):
            temp=hexarray[l+1]
            if(temp>='A'and temp<='F'):
                temp1=ord(temp)-ord('A')+10
            else:
                temp1=ord(temp)-ord('0')
            value_array[i][l]=temp1
        RGB_array[i][0]=value_array[i][0]*16+value_array[i][1]
        RGB_array[i][1]=value_array[i][2]*16+value_array[i][3]
        RGB_array[i][2]=value_array[i][4]*16+value_array[i][5]
    return RGB_array

In [119]:

def RGBhex_2RGB(rgb_hex):
#     print(rgb_hex)
    RGB=[0,0,0]
    temp_num=0
    for i in range(len(rgb_hex)):
        temp_num=0
        temp=rgb_hex[i]
        if(i!=0):
            #将字母转换为ASCII表中位置
            if(temp>='A'and temp<='F'):
                temp_num=ord(temp)-55
                #将ABCDEF转换为10_15间的数字
            else:
                #将字符数字转换为0_9间的数字
                temp_num=ord(temp)-48
            if(i%2==1):
                #根据位置乘进制
                RGB[int((i/2)-0.5)]=RGB[int((i/2)-0.5)]+16*temp_num
            else:
                RGB[int((i/2)-0.5)]=RGB[int((i/2)-0.5)]+temp_num
#     print(RGB)
    return RGB
                    
print(RGBhex_2RGB('#FEDC8A'))

[254, 220, 138]


In [135]:
import numpy as np
import json
def operate(target_color):
    sum_all=0
    with open('D:/homework/大三下信息系统设计/lipstick-gh-pages/lipstick-gh-pages/dist/lipstick.json', 'r', encoding='utf-8') as f:
        js2dic = json.load(f)
        #读取json
        brands_n=len(js2dic['brands'])
        print(brands_n)
        series_n=0
        for brands_i in range(brands_n):
            series_n=len(js2dic['brands'][brands_i]['series'])
            print("{0} has {1} series".format((js2dic['brands'][brands_i]['name']),series_n))
            for series_i in range(series_n):
                brand_name=js2dic['brands'][brands_i]['name']
                lip_name=js2dic['brands'][brands_i]['series'][series_i]['name']
                color_num=len(js2dic['brands'][brands_i]['series'][series_i]['lipsticks'])
                sum_all=color_num+sum_all
            #计算颜色总数
            print(sum_all)
        catalog=np.zeros((sum_all,4), dtype=(str,20))
        catalog_color=np.zeros((sum_all,3), dtype=int)
#         根据颜色数分配空间
        rank_color=np.zeros((sum_all,1),dtype=int)
        #catalog分为四部分：品牌名称，唇膏名称，色号id，色号值
        #将信息存入表格
        sum_i=0
        for brands_i in range(brands_n):
            series_n=len(js2dic['brands'][brands_i]['series'])
            print("brand_name",js2dic['brands'][brands_i]['name'])
            catalog[sum_i][0]=js2dic['brands'][brands_i]['name']
            for series_i in range(series_n):
                color_num=len(js2dic['brands'][brands_i]['series'][series_i]['lipsticks'])
                for color_i in range(color_num):
                    catalog[sum_i][0]=js2dic['brands'][brands_i]['name']
                    catalog[sum_i][1]=js2dic['brands'][brands_i]['series'][series_i]['name']
                    catalog[sum_i][2]=js2dic['brands'][brands_i]['series'][series_i]['lipsticks'][color_i]['name']
                    catalog[sum_i][3]=js2dic['brands'][brands_i]['series'][series_i]['lipsticks'][color_i]['id']
                    catalog_color[sum_i]=RGBhex_2RGB(js2dic['brands'][brands_i]['series'][series_i]['lipsticks'][color_i]['color'])
                    sum_i+=1
#                     print(sum_i)
        print(catalog.shape)
        RGB_distance=np.zeros((sum_all,1), dtype=float)
        for i in range(sum_all):
            #计算相似度，target是此前通过domain得到的值
            RGB_distance[i]=abs(target_color[0]-catalog_color[i][0])+abs((target_color[1]-catalog_color[i][1])*(1/5))+abs(target_color[2]-catalog_color[i][2])
        RGB_distance.tolist()
        result=sorted(range(len(RGB_distance)), key=lambda k: RGB_distance[k])
#         获得颜色最像的三只口红的（以颜色的相近度为规则排序，返回位置数据）
        print("颜色最像的三只口红及其颜色")
        result_show=[]
        for i in range(3):
            loc=result[i]
            color_show=tuple(catalog_color[loc])
            print("catalog index",catalog[loc],color_show)
# operate([155, 44, 69])
if __name__ == '__main__':
    get=get_dominant_color(cropped)
    print("获得的口红颜色{0}".format(get))
    #计算相似度，get是此前通过domain得到的值
    operate(get)

获得的口红颜色(185, 38, 122)
6
Armani has 4 series
76
YSL has 6 series
180
香奈儿可可小姐 has 4 series
240
Dior迪奥 has 7 series
371
maybelline has 4 series
446
Givenchy has 7 series
537
brand_name Armani
brand_name YSL
brand_name 香奈儿可可小姐
brand_name Dior迪奥
brand_name maybelline
brand_name Givenchy
(537, 4)
颜色最像的三只口红及其颜色
catalog index ['YSL' '纯口红' '玫红色' 'N°19'] (175, 63, 121)
catalog index ['Dior迪奥' '烈焰蓝金挚红唇膏' 'none' '755'] (177, 29, 113)
catalog index ['YSL' '纯口红' '雅紫纱蓉' 'N°07'] (201, 40, 119)
6
Armani has 4 series
76
YSL has 6 series
180
香奈儿可可小姐 has 4 series
240
Dior迪奥 has 7 series
371
maybelline has 4 series
446
Givenchy has 7 series
537
brand_name Armani
brand_name YSL
brand_name 香奈儿可可小姐
brand_name Dior迪奥
brand_name maybelline
brand_name Givenchy
(537, 4)
颜色最像的三只口红及其颜色
catalog index ['Givenchy' '高定香榭红丝绒唇膏' 'Violet Velours ' 'N°42'] (155, 44, 69)
catalog index ['Dior迪奥' '魅惑星耀唇膏' 'none' '769'] (159, 47, 71)
catalog index ['YSL' '圆管or莹亮纯魅唇膏' '蔓越莓曲奇' 'N°90'] (147, 45, 69)
